In [ ]:
import shutil
import os
import random

import numpy as np

import fiftyone as fo
import fiftyone.zoo as foz

In [ ]:
def create_tracks(dataset, dets_field):
    kps_field = dets_field+"_cntr"
    frame_kps = {}
    for sample_id, frame_numbers, frames_dets in zip(*dataset.values(["id", "frames.frame_number", "frames.%s" % dets_field])):
        print(f"sample_id={sample_id}")
        if sample_id not in frame_kps:
            frame_kps[sample_id] = {}

        for frame_number, frame_dets in zip(frame_numbers, frames_dets):
            if frame_dets is None:
                continue
                
            frame_kps[sample_id][frame_number] = fo.Keypoints(keypoints=[])
            for det in frame_dets.detections:
                tlx,tly,w,h = det.bounding_box
                center = (tlx+w/2, tly+h/2)
                idx = det.index
                if idx == None:
                    continue
                frame_kps[sample_id][frame_number].keypoints.append(fo.Keypoint(label=str(idx), points=[center]))
                        
    dataset.set_values("frames."+kps_field, frame_kps, key_field="id")

In [ ]:
DSETNAME = 'qsv-test01'
if fo.dataset_exists(DSETNAME):
    fo.delete_dataset(DSETNAME)
    
dsv = foz.load_zoo_dataset('quickstart-video',dataset_name=DSETNAME)
dsv.compute_metadata()
dsv.ensure_frames()
dsv.persistent=True

dets = dsv.values('frames.detections')
dsv.set_values('frames.dets',dets)
dsv.delete_frame_field('detections')

In [ ]:
# add keypts
create_tracks(dsv,'dets')

In [ ]:
# add tempdets
LABELS = ['hazard', 'bird', 'meteor']          
for i,s in enumerate(dsv):
    lbl1 = LABELS[i % len(LABELS)]
    lbl2 = LABELS[(i+1) % len(LABELS)]    
    td0 = fo.TemporalDetection(label=lbl1,support=[10+i,20+2*i])
    td1 = fo.TemporalDetection(label=lbl2,support=[50+i,70+2*i])
    tds = fo.TemporalDetections(detections=[td0,td1])
    s['events'] = tds
    s.save()
    


### -------- working ----------

In [ ]:
# tags
set = np.random.randint(0,2,len(dsv),dtype=np.bool)
dsv[set].tag_samples('Intruder')
set = np.random.randint(0,2,len(dsv),dtype=np.bool)
dsv[set].tag_samples('Safe')

# classes
CLASSES = ['sunny', 'rainy', 'hailing']
num_class = len(CLASSES)
for s in dsv.iter_samples(autosave=True):
    icls = np.random.randint(0,num_class)
    conf = np.random.random()
    c = fo.Classification(label=CLASSES[icls],confidence=conf)
    s['weather'] = c
    
# tempdets
CLASSES = ['vehicle','person','animal']
for s in dsv.iter_samples(autosave=True):
    nf = s.metadata.total_frame_count
    icls = np.random.randint(0,num_class)
    conf = np.random.random()
    f0 = np.random.randint(0,nf)
    f1 = np.random.randint(0,nf)
    if f0 > f1:
        tmp = f0
        f0 = f1
        f1 = tmp
    s['objects'] = fo.TemporalDetection(label=CLASSES[icls],
                                        support=[f0,f1],
                                        confidence=conf)
    
# frame classes
for s in dsv.iter_samples(autosave=True):
    for f,frm in s.frames.items():
        if np.random.random()>0.95:
            icls = np.random.randint(0,num_class)
            conf = np.random.random()
            c = fo.Classification(label=CLASSES[icls],confidence=conf)
            frm['intruder'] = c
            frm.save()
                             